In [1]:
import psycopg2
import numpy as np
import pandas as pd
import networkx as nx
from community import community_louvain
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import warnings
import math
import operator
import itertools

warnings.simplefilter(action='ignore', category=UserWarning)

# This changes on reboot -_-
wsl_ip = "172.25.148.82"

# Database connection
conn = psycopg2.connect(host=wsl_ip, port = 5432, database="postgres", user="postgres", password="postgres")

# Global average chatters
average_chatters = dict()
    

In [2]:
def getListOfChannels(limit):
    tables = pd.io.sql.read_sql_query("SELECT * FROM information_schema.tables WHERE table_schema LIKE '%public%'", conn)['table_name']
    for table in tables:
        if table in ['iterations','streams','users']:
            continue
        average_chatters[table] = pd.io.sql.read_sql_query(f"SELECT ROUND(AVG(total_chatters), 0) as avg_total FROM {table}", conn)['avg_total'][0]
    if limit == -1:
        return dict(list(dict(sorted(average_chatters.items(), key=operator.itemgetter(1),reverse=True)).items()))
    else:
        return dict(list(dict(sorted(average_chatters.items(), key=operator.itemgetter(1),reverse=True)).items())[0:limit])

In [3]:
def writeChatterCounts():
    arr = np.asarray(getListOfChannels(660).values())
    arr.tofile('counts.csv', sep=',')
    
writeChatterCounts()

In [4]:
def getAverageChatterCounts(channels):
    counts = dict()
    for channel in channels:
        df_cc = pd.io.sql.read_sql_query(f"SELECT ROUND(AVG(total_chatters), 0) as avg_chatters FROM {channel}", conn)
        counts[channel] = df_cc['avg_chatters'][0]
    return counts

In [5]:
# average chatter count for all channels is 5983.245
def buildEdgeList(channels):
    edge_list = dict()
    for channel in channels:
        df_cc = pd.io.sql.read_sql_query(f"SELECT ROUND(AVG(total_chatters), 0) as avg_chatters FROM {channel}", conn)
        #print(df_cc['avg_chatters'][0])
        if(df_cc['avg_chatters'][0] < 5000):
            continue
        
        df = pd.io.sql.read_sql_query(f"SELECT channel_name, ROUND(AVG(overlap_count), 0) as avg_overlap FROM {channel} GROUP BY channel_name", conn)
        df = df[df.channel_name.isin(channels)]
        for row in df.iterrows():
            if (row[1]['channel_name'], channel) in edge_list.keys():
                entry = edge_list[(row[1]['channel_name'], channel)]
                edge_list[(row[1]['channel_name'], channel)] = (entry + row[1]['avg_overlap']) / 2.0 # divide by 2?
            else:
                edge_list[(channel, row[1]['channel_name'])] = row[1]['avg_overlap']
    return edge_list

In [61]:
channels = getListOfChannels(660)
buildEdgeList(channels)
#print(channels)

380440.0
229511.0
173505.0
128513.0
105987.0
104564.0
99127.0
97967.0
85209.0
75484.0
75269.0
73945.0
73229.0
68950.0
63917.0
60761.0
60651.0
59969.0
58973.0
58706.0
56955.0
55910.0
55486.0
54758.0
54171.0
53897.0
52798.0
51570.0
49638.0
49275.0
48266.0
47404.0
47388.0
45421.0
44048.0
42724.0
42252.0
42240.0
40513.0
39632.0
39002.0
38850.0
38803.0
38230.0
35432.0
35367.0
31610.0
31222.0
30924.0
30297.0
29107.0
27818.0
27571.0
27410.0
27386.0
27321.0
27200.0
27105.0
26982.0
26720.0
26657.0
26574.0
26275.0
25726.0
25717.0
25590.0
25363.0
25273.0
24544.0
24161.0
24043.0
23177.0
23134.0
22508.0
22489.0
21618.0
21576.0
21110.0
20820.0
20715.0
20662.0
20579.0
20506.0
20463.0
20120.0
19969.0
19940.0
19937.0
19867.0
19592.0
19562.0
19424.0
19292.0
19141.0
19091.0
18863.0
18794.0
18757.0
18575.0
18554.0
18466.0
18402.0
18401.0
18395.0
18298.0
18215.0
18215.0
17840.0
17793.0
17729.0
17656.0
17306.0
17244.0
16886.0
16818.0
16808.0
16778.0
16758.0
16737.0
16727.0
16395.0
16388.0
16340.0
16284.0
16

KeyboardInterrupt: 

In [6]:
def buildNetwork(write_out):
    # Get top 660 channels (average chatter count greater than 4000)
    channels = getListOfChannels(660)
    graph = nx.Graph()
    for channel in channels:
        graph.add_node(channel)
    print('Done fetching channel info.')
    
    # Initial read, insertion of edges, weights/chatter count
    edge_list = buildEdgeList(channels)
    print('Done creating edge list.')
    
    # Add edges
    for key in edge_list.keys():
        chatter_count = (average_chatters[key[0]] if average_chatters[key[0]] < average_chatters[key[1]] else average_chatters[key[1]])
        edge_list[key] = round(edge_list[key] / chatter_count, 3)
        if edge_list[key] > 0.04:
            graph.add_edge(key[0], key[1], weight=edge_list[key])
    print('Done adding edges.')
    
    # Add Gephi attributes
    for channel in channels:
        graph.nodes[channel]['viz'] = {'size': average_chatters[channel]}
    
    # Write to Gephi file
    if write_out:
        nx.write_gexf(graph, "graph.gexf")
        print('Done writing Gelphi graph data.')
        
    return graph

In [12]:
def getRecommendations(graph, channel, num_recommendations):
    partitions = community_louvain.best_partition(graph, resolution=1.0)
    communities = dict()
    for key in partitions:
        communities.setdefault(partitions[key], []).append(key)
    recommendations = np.ndarray.tolist(np.unique(np.random.choice(communities[partitions[channel]], 3)))
    random_walks = [nx.pagerank_numpy(graph, personalization={start_channel: 1}) for start_channel in recommendations]

    while True:
        selection = np.random.choice(recommendations, 1)[0]
        walk = nx.pagerank_numpy(graph, personalization={selection: 1})
        highest_entries = list(dict(sorted(walk.items(), key=operator.itemgetter(1),reverse=True)[0:10]).keys())
        candidate = np.random.choice(highest_entries, 1)[0]
        if candidate == channel:
            continue
        if not candidate in recommendations:
            recommendations.append(candidate)
        if len(recommendations) >= num_recommendations:
            break

    print(recommendations)

In [8]:
graph = buildNetwork(False)


Done fetching channel info.
Done creating edge list.
Done adding edges.


In [29]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    getRecommendations(graph, 'sodapoppin', 8)

['bigbossboze', 'clintstevens', 'zoil', 'hasanabi', 'erobb221', 'xqc', 'esfandtv', 'zackrawrr']


In [ ]:
def fetchDataFrameWithCategories(channel_name):
    #query = f"SELECT {channel_name}.*, streams.category_name FROM {channel_name} INNER JOIN streams ON ({channel_name}.iteration=streams.iteration AND streams.channel_name LIKE '%{channel_name}%')"
    df = pd.io.sql.read_sql_query(f"SELECT * FROM {channel_name}", conn)
    df_category = pd.io.sql.read_sql_query(f"SELECT iteration, category_name FROM streams WHERE channel_name LIKE '%xqc%'", conn)
    category_col = []
    
    for df_row in df.iterrows():
        iteration = df_row[1]['iteration']
        categories = []
        for cat_row in df_category.iterrows():
            if iteration-24 <= cat_row[1]['iteration'] <= iteration+24:
                if not cat_row[1]['category_name'] in categories:
                    categories.append(cat_row[1]['category_name'])
                
        #print(','.join(categories))
        category_col.append(','.join(categories))
    df['channel_categories'] = category_col
    return df

In [8]:
channels = getListOfChannels(1000)
print(channels)

{'riotgames': 380440.0, 'xqc': 229511.0, 'easportsfifa': 173505.0, 'adinross': 128513.0, 'trainwreckstv': 105987.0, 'kaicenat': 104564.0, 'mizkif': 99127.0, 'primevideo': 97967.0, 'hasanabi': 85209.0, 'shroud': 75484.0, 'castro_1021': 75269.0, 'esl_csgo': 73945.0, 'zackrawrr': 73229.0, 'blastpremier': 68950.0, 'a_seagull': 63917.0, 'loltyler1': 60761.0, 'esl_csgod': 60651.0, 'lirik': 59969.0, 'tarik': 58973.0, 'twitch': 58706.0, 'austinshow': 56955.0, 'ranboolive': 55910.0, 'fextralife': 55486.0, 'nickmercs': 54758.0, 'moistcr1tikal': 54171.0, 'roshtein': 53897.0, 'amouranth': 52798.0, 'criticalrole': 51570.0, 'tenz': 49638.0, 'brucedropemoff': 49275.0, 'rocketleague': 48266.0, 'halo': 47404.0, 'sodapoppin': 47388.0, 'tfue': 45421.0, 'tsm_imperialhal': 44048.0, 'kyedae': 42724.0, 'pokimane': 42252.0, 'summit1g': 42240.0, 'ml7support': 40513.0, 'rainbow6': 39632.0, 'emiru': 39002.0, 'sinatraa': 38850.0, 'shylily': 38803.0, 'tommyinnit': 38230.0, 'redbull': 35432.0, 'aceu': 35367.0, 'nmp

In [36]:
def graph2Matrix():
    channels = getListOfChannels(20)
    edge_list = dict()
    name_dict = dict()
    matrix = [[0 for j in range(20)] for i in range(20)]
    
    i = 0
    for channel in channels.keys():
        name_dict[channel] = i
        i += 1
    
    for channel in channels:
        #print(channel)
        df = pd.io.sql.read_sql_query(f"SELECT channel_name, ROUND(AVG(overlap_count), 0) as avg_overlap FROM {channel} GROUP BY channel_name", conn)
        df = df[df.channel_name.isin(channels)]
        #print(df)
        for row in df.iterrows():
            if (row[1]['channel_name'], channel) in edge_list.keys():
                entry = edge_list[(row[1]['channel_name'], channel)]
                edge_list[(row[1]['channel_name'], channel)] = (entry + row[1]['avg_overlap']) / 2.0 # divide by 2?
            else:
                edge_list[(channel, row[1]['channel_name'])] = row[1]['avg_overlap']
                
    for edge_tuple in edge_list.keys():
        print('[', name_dict[edge_tuple[0]], ',', name_dict[edge_tuple[1]], '] :', edge_list[edge_tuple])
        matrix[name_dict[edge_tuple[0]]][name_dict[edge_tuple[1]]] = edge_list[edge_tuple]
    
    print(matrix)
    np.savetxt('output.csv',matrix,delimiter=",",fmt='%.0f')
graph2Matrix()

[ 0 , 9 ] : 4513.0
[ 0 , 16 ] : 4496.0
[ 0 , 19 ] : 2246.0
[ 0 , 5 ] : 2087.0
[ 0 , 12 ] : 4001.0
[ 0 , 7 ] : 3596.0
[ 0 , 18 ] : 3128.0
[ 0 , 1 ] : 13326.0
[ 0 , 11 ] : 8880.0
[ 0 , 8 ] : 3273.0
[ 0 , 17 ] : 6528.0
[ 0 , 6 ] : 8847.0
[ 0 , 4 ] : 7778.0
[ 0 , 15 ] : 16905.0
[ 0 , 10 ] : 2135.0
[ 0 , 14 ] : 5028.0
[ 1 , 13 ] : 2393.0
[ 1 , 9 ] : 6255.0
[ 1 , 5 ] : 11941.0
[ 1 , 12 ] : 7462.0
[ 1 , 18 ] : 3739.0
[ 1 , 11 ] : 2228.0
[ 1 , 8 ] : 16595.0
[ 1 , 17 ] : 10457.0
[ 1 , 6 ] : 28668.0
[ 1 , 4 ] : 28769.0
[ 1 , 15 ] : 9665.0
[ 1 , 10 ] : 2582.0
[ 1 , 14 ] : 6854.0
[ 2 , 14 ] : 112.0
[ 2 , 4 ] : 1217.0
[ 2 , 10 ] : 9619.0
[ 2 , 11 ] : 93.0
[ 2 , 8 ] : 289.0
[ 2 , 17 ] : 315.0
[ 3 , 5 ] : 23444.0
[ 3 , 11 ] : 87.0
[ 3 , 12 ] : 909.0
[ 4 , 18 ] : 774.0
[ 4 , 9 ] : 2066.0
[ 4 , 11 ] : 1221.0
[ 4 , 8 ] : 4242.0
[ 4 , 17 ] : 2575.0
[ 4 , 19 ] : 3243.0
[ 4 , 16 ] : 1548.0
[ 4 , 6 ] : 5036.0
[ 4 , 5 ] : 10496.0
[ 4 , 12 ] : 1927.0
[ 4 , 15 ] : 3175.0
[ 4 , 10 ] : 2169.0
[ 4 , 14 ] : 1329.0

In [35]:
matrix = [[0 for j in range(5)] for i in range(5)]
matrix[1][1] = 2
matrix[2][3] = 5
matrix[4][2] = 4
print(matrix)

[[0, 0, 0, 0, 0], [0, 2, 0, 0, 0], [0, 0, 0, 5, 0], [0, 0, 0, 0, 0], [0, 0, 4, 0, 0]]


In [ ]:
"""
# Community Detection
    print('Louvian, this will take years')
    
    partition = community_louvain.best_partition(graph)
    print(partition)
    
    #communities = girvan_newman(graph)
    #print(tuple(sorted(c) for c in next(communities)))
    print('Done with Louvian.')
    
    plt.figure(3,figsize=(64, 64)) 
    pos=nx.spring_layout(graph,center=(6,6))
    cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
    #nx.draw(graph,pos,with_labels=True,edge_color=edge_list.values(), cmap=cmap)
    labels = nx.get_edge_attributes(graph,'weight')
    nx.draw_networkx_nodes(graph, pos, partition.keys(), with_labels=True, cmap=cmap, node_color=list(partition.values()))
    nx.draw_networkx_edges(graph, pos, alpha=0.5)
    nx.draw_networkx_edge_labels(graph,pos,edge_labels=labels)
    plt.show()
"""